In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [2]:
df_train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
df_greek = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")

In [3]:
df_train.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [4]:
df_greek.head()

,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020


In [5]:
print(df_train.shape)
print(df_greek.shape)

(617, 58)
(617, 6)


In [6]:
df_train.dropna(inplace=True)

In [7]:
print(df_train.shape)

(548, 58)


In [8]:
onehot = OneHotEncoder()
ej_onehot = onehot.fit_transform(df_train["EJ"].values.reshape(-1, 1)).toarray()

In [9]:
len(ej_onehot)

548

In [10]:
X_train = df_train.drop("EJ", axis=1)

In [11]:
ej_df = pd.DataFrame(ej_onehot)

In [12]:
X_train.reset_index(drop=True, inplace=True)

In [13]:
ej_df

,0,1
0,0.0,1.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
543,1.0,0.0
544,1.0,0.0
545,0.0,1.0
546,1.0,0.0


In [14]:
X_train = pd.concat([X_train, ej_df], ignore_index=True, axis=1)

In [15]:
X_train.iloc[:, 56]

0      1
1      0
2      0
3      0
4      1
      ..
543    0
544    0
545    0
546    0
547    0
Name: 56, Length: 548, dtype: int64

In [16]:
y_train = X_train[56]
X_train.drop(56, axis=1, inplace=True)

In [17]:
X_train = X_train.iloc[:, 1:]

In [18]:
pca = PCA(n_components=None)
X_train_trans = pca.fit_transform(X_train.iloc[:, 1:])
pca.explained_variance_

array([4.03812671e+08, 1.36695159e+08, 6.44303145e+07, 1.29036899e+07,
       5.03220306e+06, 2.95462869e+06, 1.94790180e+06, 1.76394338e+05,
       9.58522867e+04, 3.54172337e+04, 3.07331438e+04, 2.29826063e+04,
       1.55939465e+04, 1.48514831e+04, 8.34916403e+03, 6.36107665e+03,
       4.61204305e+03, 3.35769236e+03, 2.81590661e+03, 2.03165165e+03,
       1.57757600e+03, 1.18143743e+03, 1.07851286e+03, 6.84462462e+02,
       6.05829560e+02, 3.41912380e+02, 2.92706955e+02, 1.87935727e+02,
       1.15231254e+02, 9.78097899e+01, 8.63609128e+01, 6.51443388e+01,
       5.97651670e+01, 4.32811611e+01, 3.73982425e+01, 2.21544068e+01,
       1.46477108e+01, 1.14781001e+01, 1.06515968e+01, 8.92448190e+00,
       6.57108446e+00, 3.52390137e+00, 2.88910957e+00, 1.76200658e+00,
       1.63713009e+00, 1.21246365e+00, 1.85134634e-01, 1.30366894e-01,
       9.72522523e-02, 7.96784883e-02, 5.15996856e-02, 3.84983363e-02,
       1.73425925e-02, 7.08526035e-03, 5.90075556e-05, 3.73205527e-24])

In [19]:
explained_var = pca.explained_variance_

In [20]:
pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=40), LogisticRegression(random_state=1, solver='lbfgs'))

In [21]:
y_train.values.flatten()

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
pipe_lr.fit(X_train, y_train.values.flatten())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=40)),
                ('logisticregression', LogisticRegression(random_state=1))])

In [23]:
df_test = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")

In [24]:
df_test

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
onehot = OneHotEncoder()
ej_onehot = onehot.fit_transform(df_test["EJ"].values.reshape(-1, 1)).toarray()

In [26]:
X_test = df_test.drop("EJ", axis=1)

In [27]:
ej_df = pd.DataFrame(ej_onehot)

In [28]:
if len(ej_df.columns) < 3:
    ej_df[1] = 0

In [29]:
ej_df

,0,1
0,1.0,0
1,1.0,0
2,1.0,0
3,1.0,0
4,1.0,0


In [30]:
X_test.reset_index(drop=True, inplace=True)

In [31]:
X_test = pd.concat([X_test, ej_df], ignore_index=True, axis=1)

In [32]:
X_test = X_test.iloc[:, 1:]

In [33]:
X_test

,1,2,3,4,5,6,7,8,9,10,...,48,49,50,51,52,53,54,55,56,57
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [34]:
output = pipe_lr.predict_proba(X_test)

In [35]:
df_test["class_0"] = output[:, 0]

In [36]:
df_test["class_1"] = output[:, 1]

In [37]:
output = df_test[["Id", "class_0", "class_1"]]

In [38]:
output.to_csv('submission.csv', index=False)

In [39]:
output

,Id,class_0,class_1
0,00eed32682bb,0.08805,0.91195
1,010ebe33f668,0.08805,0.91195
2,02fa521e1838,0.08805,0.91195
3,040e15f562a2,0.08805,0.91195
4,046e85c7cc7f,0.08805,0.91195
